In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, avg, stddev, min, max, percentile_approx, mode, countDistinct

# Paso 1: Configurar PySpark y cargar los datos
# Clonar el repositorio
!git clone https://github.com/Legelica/TFM-UNIR-LAVP

Cloning into 'TFM-UNIR-LAVP'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 1.08 MiB | 3.68 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [14]:
# Leer CSV desde la carpeta clonada
spark = SparkSession.builder.appName("EDA_Turismo").getOrCreate()

# Ruta base donde están los archivos
ruta_base = "/content/TFM-UNIR-LAVP/Dataset for TFM/"

# Lista de archivos a leer (solo los nombres, sin la ruta completa)
archivos = {
    "ANP Administracion Nacional Definitiva-dataset.csv": ";",
    "Distribución gastos mantener ANP.csv": ";",
    "Empresas de servicios turisticos en Peru registradas por IPERU.csv": ";",
    "Frecuencia de visitas turisticas en las Areas Naturales Protegidas por el estado.csv": ";",
    "Ingresos recaudacion.csv": ";",
    "Inventario Nacional de Recursos Turisticos.csv": ";",
    "Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv": ","
}

# Crear un diccionario para almacenar los DataFrames
dfs = {}

# Leer y guardar cada archivo en el diccionario
for i, (nombre_archivo, delimitador) in enumerate(archivos.items(), start=1):
    ruta_completa = f"{ruta_base}{nombre_archivo}"
    df = spark.read.csv(ruta_completa, header=True, inferSchema=True, sep=delimitador)
    dfs[f"df{i}"] = df
    print(f"- Mostrando datos de {archivo} -> Guardado en dfs['df{i}']\n")

- Mostrando datos de Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv -> Guardado en dfs['df1']

- Mostrando datos de Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv -> Guardado en dfs['df2']

- Mostrando datos de Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv -> Guardado en dfs['df3']

- Mostrando datos de Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv -> Guardado en dfs['df4']

- Mostrando datos de Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv -> Guardado en dfs['df5']

- Mostrando datos de Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv -> Guardado en dfs['df6']

- Mostrando datos de Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv -> Guardado en dfs['df7']



In [16]:
# Paso 2: Inspección de estructura de los datos
print("\n=== Inspección de la estructura de los datos ===")
for key, df in dfs.items():
    print(f"\nEstructura de {key}:")
    df.printSchema()
    print(f"Ejemplo de datos de {key}:")
    df.show(5)


=== Inspección de la estructura de los datos ===

Estructura de df1:
root
 |-- FECHA_CORTE: integer (nullable = true)
 |-- ANP_CATE: string (nullable = true)
 |-- ANP_CODI: string (nullable = true)
 |-- ANP_NOMB: string (nullable = true)
 |-- ANP_SECT: string (nullable = true)
 |-- ANP_UBPO: string (nullable = true)
 |-- ANP_SULEG: double (nullable = true)
 |-- ANP_UICN: string (nullable = true)
 |-- ANP_BALEC: string (nullable = true)
 |-- ANP_FELEC: integer (nullable = true)
 |-- ANP_BALEM: string (nullable = true)
 |-- ANP_FELEM: integer (nullable = true)
 |-- ANP_WDPAID: string (nullable = true)
 |-- UBIGEO1: integer (nullable = true)
 |-- DEPARTAMENTO1: string (nullable = true)
 |-- PROVINCIA1: string (nullable = true)
 |-- DISTRITO1: string (nullable = true)
 |-- UBIGEO2: integer (nullable = true)
 |-- DEPARTAMENTO2: string (nullable = true)
 |-- PROVINCIA2: string (nullable = true)
 |-- DISTRITO2: string (nullable = true)
 |-- UBIGEO3: string (nullable = true)
 |-- DEPARTAMENTO

In [17]:
# Paso 3: Revisión de calidad de los datos (valores nulos, duplicados)
print("\n=== Revisión de calidad de los datos ===")
for key, df in dfs.items():
    print(f"\nAnálisis de {key}:")

    # Contar valores nulos por columna
    null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
    print("Valores nulos por columna:")
    null_counts.show()

    # Contar valores duplicados
    duplicated_count = df.count() - df.distinct().count()
    print(f"Cantidad de registros duplicados en {key}: {duplicated_count}")


=== Revisión de calidad de los datos ===

Análisis de df1:
Valores nulos por columna:
+-----------+--------+--------+--------+--------+--------+---------+--------+---------+---------+---------+---------+----------+-------+-------------+----------+---------+-------+-------------+----------+---------+-------+-------------+----------+---------+-------+-------------+----------+---------+-------+-------------+----------+---------+-------+-------------+----------+---------+-------+-------------+----------+---------+-------+-------------+----------+---------+-------+-------------+----------+---------+--------+--------------+-----------+----------+--------+--------------+-----------+----------+--------+--------------+-----------+----------+--------+--------------+-----------+----------+--------+--------------+-----------+----------+--------+--------------+-----------+----------+--------+--------------+-----------+----------+--------+--------------+-----------+----------+--------+-------------

In [18]:
# Paso 4: Estadísticas descriptivas
print("\n=== Estadísticas descriptivas ===")
for key, df in dfs.items():
    print(f"\nEstadísticas de {key}:")
    df.describe().show()

    # Cálculo de percentiles (25%, 50%, 75%)
    numeric_cols = [c for c, t in df.dtypes if t in ("int", "double")]
    for col_name in numeric_cols:
        percentiles = df.select(
            percentile_approx(col(col_name), [0.25, 0.5, 0.75]).alias("percentiles")
        ).collect()[0]["percentiles"]
        print(f"Percentiles (25%, 50%, 75%) para {col_name}: {percentiles}")


=== Estadísticas descriptivas ===

Estadísticas de df1:
+-------+-----------+--------------------+--------+--------------------+-------------+--------------------+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+----------+-----------------+------------------+-------------+----------+----------+------------------+-------------+----------+---------+------------------+-------------+----------+---------+------------------+-------------+----------+------------+------------------+-------------+----------+---------+-----------------+-------------+----------+-----------+-----------------+-------------+----------+----------+------------------+-------------+----------+-------------+-----------------+--------------+-----------+----------+-----------------+--------------+-------------+----------+-----------------+--------------+-----------+-------------+-----------------

In [21]:
# Paso 5: Análisis de relaciones entre variables
print("\n=== Análisis de relaciones entre variables ===")
for key, df in dfs.items():
    print(f"\nRelaciones en {key}:")

    # Contar valores únicos de variables categóricas
    categorical_cols = [c for c, t in df.dtypes if t == "string"]
    for col_name in categorical_cols:
        unique_values = df.select(countDistinct(col(col_name))).collect()[0][0]
        print(f"Número de valores únicos en {col_name}: {unique_values}")


=== Análisis de relaciones entre variables ===

Relaciones en df1:
Número de valores únicos en ANP_CATE: 9
Número de valores únicos en ANP_CODI: 91
Número de valores únicos en ANP_NOMB: 90
Número de valores únicos en ANP_SECT: 5
Número de valores únicos en ANP_UBPO: 34
Número de valores únicos en ANP_UICN: 5
Número de valores únicos en ANP_BALEC: 57
Número de valores únicos en ANP_BALEM: 8
Número de valores únicos en ANP_WDPAID: 67
Número de valores únicos en DEPARTAMENTO1: 23
Número de valores únicos en PROVINCIA1: 62
Número de valores únicos en DISTRITO1: 81
Número de valores únicos en DEPARTAMENTO2: 22
Número de valores únicos en PROVINCIA2: 43
Número de valores únicos en DISTRITO2: 52
Número de valores únicos en UBIGEO3: 35
Número de valores únicos en DEPARTAMENTO3: 16
Número de valores únicos en PROVINCIA3: 29
Número de valores únicos en DISTRITO3: 34
Número de valores únicos en DEPARTAMENTO4: 16
Número de valores únicos en PROVINCIA4: 24
Número de valores únicos en DISTRITO4: 26

In [22]:
# Paso 6: Preparación de los datos
print("\n=== Preparación de los datos ===")
for key, df in dfs.items():
    print(f"\nProcesando {key}...")

    # Eliminar duplicados
    df = df.dropDuplicates()

    # Imputar valores nulos con la media en variables numéricas
    numeric_cols = [c for c, t in df.dtypes if t in ("int", "double")]
    for col_name in numeric_cols:
        mean_value = df.select(avg(col(col_name))).collect()[0][0]
        df = df.fillna({col_name: mean_value})

    # Reemplazar valores nulos en categóricas con 'Desconocido'
    categorical_cols = [c for c, t in df.dtypes if t == "string"]
    df = df.fillna({col_name: "Desconocido" for col_name in categorical_cols})

    # Guardar el DataFrame limpio en el diccionario
    dfs[key] = df
    print(f"Datos de {key} preparados correctamente.")



=== Preparación de los datos ===

Procesando df1...
Datos de df1 preparados correctamente.

Procesando df2...
Datos de df2 preparados correctamente.

Procesando df3...
Datos de df3 preparados correctamente.

Procesando df4...
Datos de df4 preparados correctamente.

Procesando df5...
Datos de df5 preparados correctamente.

Procesando df6...
Datos de df6 preparados correctamente.

Procesando df7...
Datos de df7 preparados correctamente.


In [27]:
print("\n=== EDA COMPLETO ===")


=== EDA COMPLETO ===


In [33]:
#Guardar los dataset procesados
# Ruta donde se guardarán los archivos procesados
ruta_salida = "/content/TFM-UNIR-LAVP/Procesados/"

# Diccionario con los nombres correctos para guardar
nombres_guardado = {
    "df1": "ANP Administracion Nacional.csv",
    "df2": "Distribución gastos mantener ANP.csv",
    "df3": "Empresas de servicios turisticos en Peru.csv",
    "df4": "Frecuencia de visitas turisticas en las Areas Naturales Protegidas.csv",
    "df5": "Ingresos recaudacion.csv",
    "df6": "Inventario Nacional de Recursos Turisticos.csv",
    "df7": "Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv"
}

# Guardar cada DataFrame con su nombre correspondiente
for nombre_df, nombre_archivo in nombres_guardado.items():
    if nombre_df in dfs:
        nombre_salida = f"{ruta_salida}{nombre_archivo}"
        dfs[nombre_df].write.mode("overwrite") \
                      .option("header", "true") \
                      .option("charset", "ISO-8859-1") \
                      .csv(nombre_salida)
        print(f"-> Guardado: {nombre_salida}")

-> Guardado: /content/TFM-UNIR-LAVP/Procesados/ANP Administracion Nacional.csv
-> Guardado: /content/TFM-UNIR-LAVP/Procesados/Distribución gastos mantener ANP.csv
-> Guardado: /content/TFM-UNIR-LAVP/Procesados/Empresas de servicios turisticos en Peru.csv
-> Guardado: /content/TFM-UNIR-LAVP/Procesados/Frecuencia de visitas turisticas en las Areas Naturales Protegidas.csv
-> Guardado: /content/TFM-UNIR-LAVP/Procesados/Ingresos recaudacion.csv
-> Guardado: /content/TFM-UNIR-LAVP/Procesados/Inventario Nacional de Recursos Turisticos.csv
-> Guardado: /content/TFM-UNIR-LAVP/Procesados/Registro de monitoreo de la Deforestacion en el ambito de las Areas Naturales Protegidas.csv


In [34]:
#Descargar carpeta
import shutil
from google.colab import files

# Comprimir la carpeta
shutil.make_archive('/content/Dataset-Procesados', 'zip', '/content/TFM-UNIR-LAVP/Procesados')

# Descargar el archivo ZIP
files.download('/content/Dataset-Procesados.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>